# 🎓 **Applied Artificial Intelligence Master**

## 🤖 **Course: Fintech and Digital Innovation in Finance (Gpo 10)**

### 🏛️ Tecnológico de Monterrey

#### 👨‍🏫 **Lead Professor:** Marie-Ève Malette
#### 👩‍🏫 **Assistant Professor:** Yetnalezi Quintas Ruiz

### 📊 **Project 3 |  Retrieval Augmented Generation with S&P 500 news **

#### 📅 **August 9th, 2025**

#### **Student:**
* 🧑‍💻 Oscar Enrique García García (A01016093)

# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [1]:
%pip install sentence-transformers
%pip install faiss-cpu
%pip install openai
%pip install langchain-openai

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from openai import OpenAI
import yfinance as yf
from typing import List
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.retrievers import BaseRetriever
from langchain_openai import ChatOpenAI 
from typing import List, Optional
import time
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

/Users/oscar.garcia@digitalfemsa.com/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [3]:
K = 25

In [4]:
df_news = pd.read_csv('df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [5]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.
 

In [7]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'], 
        'TICKER': row['TICKER'], 
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [8]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [9]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [10]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [11]:
client = OpenAI()

In [12]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

As the moonlight sparkled on her shimmering coat, the unicorn whispered to the stars her dreams of adventure, knowing that each twinkle would carry her wishes far and wide.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [13]:
tickers = df_news["TICKER"].dropna().unique().tolist()

meta_records = []

for ticker in tickers:
    try:
        info = yf.Ticker(ticker).get_info()
        meta_records.append({
            "TICKER": ticker,
            "COMPANY_NAME": info.get("longName") or info.get("shortName"),
            "SECTOR": info.get("sector"),
            "INDUSTRY": info.get("industry")
        })
    except Exception as e:
        print(f"⚠️ Could not fetch info for {ticker}: {e}")
        meta_records.append({
            "TICKER": ticker,
            "COMPANY_NAME": None,
            "SECTOR": None,
            "INDUSTRY": None
        })

df_meta = pd.DataFrame(meta_records)
df_meta

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
...,...,...,...,...
485,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric
486,XYL,Xylem Inc.,Industrials,Specialty Industrial Machinery
487,YUM,"Yum! Brands, Inc.",Consumer Cyclical,Restaurants
488,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,Medical Devices


In [14]:
len(tickers)

490

## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [15]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [16]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [17]:
#Faiss Store Retriever

class FaissStoreRetriever(BaseRetriever):

    store: any  # your FaissVectorStore type
    k: int = 5
    metadata_filter: Optional[callable] = None

    def _get_relevant_documents(self, query: str) -> List[Document]:
        results = self.store.search(query, k=self.k, metadata_filter=self.metadata_filter)
        docs = []
        for doc_text, meta, score in results:
            m = dict(meta)
            m["similarity"] = float(score)
            docs.append(Document(page_content=doc_text, metadata=m))
        return docs

retriever = FaissStoreRetriever(store=faiss_store, k=5)



/Users/oscar.garcia@digitalfemsa.com/Library/Python/3.9/lib/python/site-packages/pydantic/_internal/_generate_schema.py:628: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/oscar.garcia@digitalfemsa.com/Library/Python/3.9/lib/python/site-packages/pydantic/_internal/_generate_schema.py:628: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [18]:
# Prompt Definition

prompt = ChatPromptTemplate.from_template(
    """You are a financial research assistant. Use only the provided context to answer.
If the context is insufficient, say so. Cite sources as [Doc i], where i is the order given.

Question: {question}

Context:
{context}

Answer (concise, factual, with [Doc i] citations):"""
)

In [19]:
# Function to format the documents for display

def format_docs(docs: List[Document], max_chars=600):
    out = []
    for i, d in enumerate(docs, 1):
        txt = d.page_content.replace("\n", " ").strip()
        if len(txt) > max_chars:
            txt = txt[:max_chars] + "…"
        meta = d.metadata
        meta_str = f"(Ticker: {meta.get('TICKER')}, Provider: {meta.get('PROVIDER')}, Date: {meta.get('PUBLICATION_DATE')})"
        out.append(f"[Doc {i}] {txt}\n{meta_str}")
    return "\n\n".join(out) if out else "No context found."

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)



In [20]:
# This function wraps retrieval + snippet formatting into a single step

def prepare_context(inputs):
    docs = inputs["docs"]
    context_text = format_docs(docs)
    # Extract snippet text for return
    snippets = [
        {
            "text": d.page_content,
            "metadata": d.metadata
        }
        for d in docs
    ]
    return {
        "question": inputs["question"],
        "context": context_text,
        "snippets": snippets
    }

rag_chain = (
    {"question": RunnablePassthrough(), "docs": retriever}
    | RunnableLambda(prepare_context) 
    | {
        "answer": ({"question": RunnablePassthrough(), "context": RunnablePassthrough()}
                   | prompt
                   | llm
                   | StrOutputParser()),
        "snippets": RunnableLambda(lambda x: x["snippets"])
      }
)

In [21]:
#Function to convert snippets to Markdown format

def snippets_to_markdown(snippets):
    if not snippets:
        return "_No supporting snippets found._"
    lines = []
    for i, sn in enumerate(snippets, 1):
        meta = sn["metadata"] or {}
        ticker = meta.get("TICKER", "—")
        provider = meta.get("PROVIDER", "—")
        date = meta.get("PUBLICATION_DATE", "—")
        sim = meta.get("similarity")
        sim_txt = f", sim={sim:.3f}" if isinstance(sim, (int, float)) else ""
        # Trim long text for readability
        text = sn["text"].replace("\n", " ").strip()
        if len(text) > 600:
            text = text[:600] + "…"
        lines.append(f"**[Doc {i}]** *(Ticker: {ticker}, Provider: {provider}, Date: {date}{sim_txt})*\n\n> {text}\n")
    return "\n".join(lines)

# Function to run a list of questions and print answers in Markdown format

def run_question_list(question_list, section_title):
    print_markdown(f"# {section_title}")
    for q in question_list:
        out = rag_chain.invoke(q)
        answer_md = f"**Q:** {q}\n\n**A:** {out['answer']}"
        sources_md = f"\n\n**Snippets used:**\n\n{snippets_to_markdown(out['snippets'])}"
        print_markdown(answer_md + sources_md)

In [22]:
# ---- Execute for each category ----
run_question_list(questions_topic, "Topic Questions")
run_question_list(questions_company, "Company Questions")
run_question_list(questions_industry, "Industry Questions")

# Topic Questions

**Q:** What are the major concerns expressed in financial news about inflation?

**A:** The major concerns expressed in financial news about inflation include:

1. Persistent US inflation and its potential to lead to an economic slowdown, as highlighted in the Federal Reserve's May policy meeting [Doc 1].
2. Food inflation dampening hopes for a rate cut, indicating that rising food prices are impacting monetary policy considerations [Doc 2], [Doc 3], [Doc 4], [Doc 5].

**Snippets used:**

**[Doc 1]** *(Ticker: BLK, Provider: Yahoo Finance UK, Date: 2025-05-29, sim=0.577)*

> Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.

**[Doc 2]** *(Ticker: TSLA, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 3]** *(Ticker: NVDA, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 4]** *(Ticker: LULU, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 5]** *(Ticker: AVGO, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead


**Q:** How is investor sentiment described in recent financial headlines?

**A:** Investor sentiment in recent financial headlines is characterized by a mix of optimism and skepticism. Many articles highlight ambitious price targets and bullish outlooks from Wall Street, suggesting attractive upside potential for certain stocks. However, there is a cautionary tone regarding the reliability of these forecasts due to institutional pressures on analysts, which may lead to overly optimistic predictions [Doc 1], [Doc 3], [Doc 5]. Additionally, some stocks are noted for their strong performance driven by positive catalysts, yet the sentiment remains tempered by the understanding that analysts often avoid issuing negative ratings to maintain business relationships [Doc 2], [Doc 4].

**Snippets used:**

**[Doc 1]** *(Ticker: KMX, Provider: StockStory, Date: 2025-05-26, sim=0.612)*

> 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts.

**[Doc 2]** *(Ticker: MCHP, Provider: StockStory, Date: 2025-05-20, sim=0.598)*

> 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase.

**[Doc 3]** *(Ticker: MPWR, Provider: StockStory, Date: 2025-05-06, sim=0.589)*

> 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory.

**[Doc 4]** *(Ticker: DRI, Provider: StockStory, Date: 2025-05-21, sim=0.577)*

> 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory.

**[Doc 5]** *(Ticker: RVTY, Provider: StockStory, Date: 2025-05-23, sim=0.570)*

> 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover.


**Q:** What role is artificial intelligence playing in recent finance-related news stories?

**A:** Artificial intelligence (AI) is playing a significant role in recent finance-related news stories by enhancing productivity, reducing human error, and driving investment opportunities. For instance, Jack Henry & Associates is integrating AI-driven lending technology to improve efficiency in financial services [Doc 1]. Additionally, companies like Meta Platforms are investing in various AI applications, which could potentially increase their stock value beyond their current legacy business valuations [Doc 2]. 

Billionaire hedge fund managers are also investing in AI stocks such as Palantir and Upstart, with analysts predicting substantial upside due to their AI capabilities in government and commercial sectors, as well as credit risk assessment [Doc 3]. Palantir, in particular, has seen significant stock gains as it helps clients integrate generative AI into their operations [Doc 4]. Furthermore, generative AI is recognized for its potential to streamline tasks in content production, as seen with companies like Netflix [Doc 5].

**Snippets used:**

**[Doc 1]** *(Ticker: JKHY, Provider: Insider Monkey, Date: 2025-03-17, sim=0.697)*

> Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]

**[Doc 2]** *(Ticker: META, Provider: Motley Fool, Date: 2025-05-31, sim=0.626)*

> This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.

**[Doc 3]** *(Ticker: PLTR, Provider: Motley Fool, Date: 2025-05-31, sim=0.619)*

> Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very …

**[Doc 4]** *(Ticker: PLTR, Provider: Motley Fool, Date: 2025-05-31, sim=0.618)*

> Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago.

**[Doc 5]** *(Ticker: NFLX, Provider: Motley Fool, Date: 2025-05-29, sim=0.580)*

> 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.


# Company Questions

...

**Q:** What financial news headlines connect Amazon with automation or logistics?

**A:** The context provided does not contain specific financial news headlines that directly connect Amazon with automation or logistics. However, there are mentions of Amazon's use of artificial intelligence in its operations, which may relate to automation indirectly. For instance, in [Doc 2], Matt Garman, CEO of Amazon Web Services, discusses how every aspect of Amazon is leveraging artificial intelligence. Additionally, [Doc 3] mentions automation in the context of Woodward's earnings growth, but it does not specifically link to Amazon. Therefore, the context is insufficient to provide direct headlines connecting Amazon with automation or logistics.

**Snippets used:**

**[Doc 1]** *(Ticker: TFC, Provider: Insider Monkey, Date: 2025-05-25, sim=0.601)*

> Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […]

**[Doc 2]** *(Ticker: AMZN, Provider: Bloomberg, Date: 2025-05-30, sim=0.523)*

> Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology."

**[Doc 3]** *(Ticker: TFC, Provider: MT Newswires, Date: 2025-05-23, sim=0.516)*

> Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi

**[Doc 4]** *(Ticker: AON, Provider: Zacks, Date: 2025-04-30, sim=0.513)*

> Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS).

**[Doc 5]** *(Ticker: CHRW, Provider: StockStory, Date: 2025-05-23, sim=0.503)*

> Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW).


# Industry Questions

**Q:** What are the main themes emerging in financial news about the semiconductor industry?

**A:** The main themes emerging in financial news about the semiconductor industry include:

1. **International Revenue Trends**: There is a focus on the international revenue trends of ON Semiconductor Corp. and how these trends impact Wall Street's forecasts and the stock's future performance [Doc 1].

2. **Investor Attention**: ON Semiconductor is attracting significant attention from investors, indicating a growing interest in its stock and potential future developments [Doc 2].

3. **Earnings Performance**: Despite soft earnings reports, there appears to be an optimistic sentiment among shareholders regarding ON Semiconductor, suggesting resilience in investor confidence [Doc 3].

4. **Market Assessment**: The end of the earnings season is seen as an opportunity to evaluate how various semiconductor companies, including Vishay Intertechnology, are navigating the current business environment [Doc 4].

5. **Stock Picks by Influential Investors**: ON Semiconductor is highlighted as one of the stock picks by billionaire Glenn Russell Dubin, indicating its perceived potential for significant upside [Doc 5]. 

These themes reflect a mix of financial performance analysis, investor sentiment, and market opportunities within the semiconductor sector.

**Snippets used:**

**[Doc 1]** *(Ticker: ON, Provider: Zacks, Date: 2025-05-13, sim=0.643)*

> Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock.

**[Doc 2]** *(Ticker: ON, Provider: Zacks, Date: 2025-05-21, sim=0.582)*

> ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store.

**[Doc 3]** *(Ticker: ON, Provider: Simply Wall St., Date: 2025-05-12, sim=0.565)*

> Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week...

**[Doc 4]** *(Ticker: ADI, Provider: StockStory, Date: 2025-05-29, sim=0.548)*

> Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1.

**[Doc 5]** *(Ticker: ON, Provider: Insider Monkey, Date: 2025-05-11, sim=0.528)*

> ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […]


**Q:** What trends are being reported in the retail industry?

**A:** The retail industry is currently experiencing several notable trends:

1. **Adaptation to Technology**: Retailers are modifying their business models in response to technological changes that affect consumer shopping habits. This adaptation is crucial as demand remains volatile, influenced by fluctuations in consumer spending [Doc 1].

2. **Impact of Economic Cycles**: The performance of consumer discretionary businesses is closely tied to economic cycles. Recently, demand trends have been unfavorable, with the industry experiencing a decline of 12.3% over the past six months, which is significantly worse than the S&P 500's 2.1% decline [Doc 3].

3. **Supply Chain Adjustments**: In light of ongoing trade tensions and tariff policies, retail executives are shifting their supply chains. Many price increases have already been implemented in response to these changes [Doc 2].

Overall, the retail sector is facing challenges due to economic conditions and the need to adapt to technological advancements and trade-related issues.

**Snippets used:**

**[Doc 1]** *(Ticker: KMX, Provider: StockStory, Date: 2025-05-12, sim=0.590)*

> 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.

**[Doc 2]** *(Ticker: BBY, Provider: The Wall Street Journal, Date: 2025-05-29, sim=0.506)*

> Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.

**[Doc 3]** *(Ticker: HLT, Provider: StockStory, Date: 2025-05-22, sim=0.500)*

> 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline.

**[Doc 4]** *(Ticker: PKG, Provider: Simply Wall St., Date: 2025-04-26, sim=0.476)*

> Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst...

**[Doc 5]** *(Ticker: APD, Provider: Simply Wall St., Date: 2025-05-14, sim=0.476)*

> Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to...


**Q:** What risks or challenges are discussed in recent news about the energy industry?

**A:** Recent news about the energy industry highlights several risks and challenges:

1. **Legislative Risks for Renewable Energy**: A bill advancing in Congress could significantly harm the renewable energy sector by repealing crucial subsidies, making many projects uneconomical. This has led to a sharp decline in renewable energy stocks as companies had just increased production and generation capabilities [Doc 1][Doc 2].

2. **Oilfield Service Challenges**: Companies in the oilfield service sector, such as SLB, HAL, and BKR, are facing difficulties due to sliding oil prices, rising tariffs, and shrinking drilling budgets. There are concerns about whether demand from LNG and AI can provide sufficient support to counter these challenges [Doc 3][Doc 4].

3. **Tariff Uncertainties**: The ongoing trade conflicts and tariff wars are creating an unpredictable landscape for the energy sector, particularly affecting companies reliant on critical materials like copper [Doc 5]. 

These factors collectively pose significant risks to both renewable and traditional energy sectors.

**Snippets used:**

**[Doc 1]** *(Ticker: NEE, Provider: Motley Fool, Date: 2025-05-23, sim=0.536)*

> Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

**[Doc 2]** *(Ticker: ENPH, Provider: Motley Fool, Date: 2025-05-23, sim=0.536)*

> Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

**[Doc 3]** *(Ticker: HAL, Provider: Zacks, Date: 2025-05-21, sim=0.482)*

> Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

**[Doc 4]** *(Ticker: BKR, Provider: Zacks, Date: 2025-05-21, sim=0.482)*

> Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

**[Doc 5]** *(Ticker: FCX, Provider: Motley Fool, Date: 2025-05-21, sim=0.478)*

> 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.


## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance** 
2. **Completeness**  
3. **Bias or Noise** 
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

**Answer:** I think that, in general, the snippets support the generated answers by the LLM; on the other hand, the LLM is correctly saying that it doesn't have enough context to answer some questions, when it doesn't.

One thing that I could notice is that for some questions (e.g. the first question), some articles are duplicated; I think that this can mislead users to innacurate conclusiones due to the repetition of a specific topic (i.e. relavance based on repetition and not in precision).

#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

**Answer:** Besides the questions that were not fully answered, the RAG model is fully addressing the questions on the different topics. However, I can see that some answers are more detailed than others, maybe because the retrieved documents are more relevant and complete in some cases. One example of this is the first question where the articles are duplicated, so I think that this caused that the answer is reduced to only two key themes: food and persistent us inflation.

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

**Answer:** Yes, as I said in previous questions, one example of misleading snippets is the first question where articles are duplicated. This can affect the diversity or detail of the answer, centering only in one key theme (or a few). 

On the other hand, I could see that in some cases (e.g. Amazon question about automation and logistics) the retrieved documents are not even talking about Amazon or automation and logistics. This can also cause misleading or inaccurate conclusions; however, the model is correctly saying that it doesn't have enough context to answer in a trustworthy way.


#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

**Answer:** I couldn't detect some conflicting viewpoints between the different retrieved snippets in the questions. I think that the model is showing consistent information in all the answers. Maybe one case could be the one talking about the investor sentiment described in recent financial headlines; however, the model is just summarizing what the articles say: there is positive sentiment, but it should be treated with skepticism.

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

**Answer:** Some of the improvements I would include are:

- Deduplication: Before retrieving the top-k news, I would include a function that depluicates the articles (repeated on many tickers). This could give us more detailed information related with every questions, without giving more weight to specific terms only because the duplicated articles.

- Filtering: When doing questions about a specific company (e.g. Amazon), include filters to only retrieve articles of that specific ticker.

- Prompting: Include some instructions to delimit the retrieved documents to talk about only for the involved company (explicit guardrails) and to avoid redundancy.

- Adjusting k: I think that a k of 5 could be a little bit small to answer accurately one of the formulated questions. Also, maybe I would use a different k for every type of question; I think that some topics need more articles than others.
  

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [23]:
len(tickers)

490

In [24]:
rows = []
for t in tickers:
    try:
        tk = yf.Ticker(t)
        info = tk.info 
        rows.append({
            "TICKER": t,
            "COMPANY": info.get("longName"),
            "SECTOR": info.get("sector"),
            "INDUSTRY": info.get("industry")
        })
        time.sleep(0.5)
    except Exception as e:
        print(f"⚠️ Error fetching {t}: {e}")
        rows.append({"TICKER": t, "COMPANY": None, "SECTOR": None, "INDUSTRY": None})

df_meta = pd.DataFrame(rows)

In [25]:
df_news_md = df_news.merge(df_meta, on="TICKER", how="left")
df_news_md.head()

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL,EMBEDDED_TEXT,COMPANY,SECTOR,INDUSTRY
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...,2 Dow Jones Stocks with Promising Prospects an...,3M Company,Industrials,Conglomerates
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...,3 S&P 500 Stocks Skating on Thin Ice : The S&P...,3M Company,Industrials,Conglomerates
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...,3M Rises 15.8% YTD: Should You Buy the Stock N...,3M Company,Industrials,Conglomerates
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,3M Company,Industrials,Conglomerates
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...,3 Cash-Producing Stocks with Questionable Fund...,3M Company,Industrials,Conglomerates


In [26]:
ticker_lookup = (
    df_meta.set_index("TICKER")[["COMPANY", "SECTOR", "INDUSTRY"]]
    .to_dict(orient="index")
)

In [27]:
def enrich_meta(meta, lookup):
    """Attach COMPANY, SECTOR, INDUSTRY from ticker_lookup (non-destructive)."""
    m = dict(meta or {})
    t = m.get("TICKER")
    if t and t in lookup:
        for key, val in lookup[t].items():
            if val is not None:
                m[key] = val
    return m

def format_docs_v2(docs, lookup, max_chars=600):
    """Same idea as format_docs, but prints richer metadata into the prompt context."""
    if not docs:
        return "No context found."
    out = []
    for i, d in enumerate(docs, 1):
        txt = d.page_content.replace("\n", " ").strip()
        if len(txt) > max_chars:
            txt = txt[:max_chars] + "…"
        m = enrich_meta(d.metadata, lookup)
        ticker = m.get("TICKER", "—")
        company = m.get("COMPANY", "—")
        sector = m.get("SECTOR", "—")
        industry = m.get("INDUSTRY", "—")
        provider = m.get("PROVIDER", "—")
        date = m.get("PUBLICATION_DATE", "—")
        sim = m.get("similarity")
        sim_txt = f", sim={sim:.3f}" if isinstance(sim, (int, float)) else ""
        out.append(
            f"[Doc {i}] {txt}\n"
            f"(Ticker: {ticker}, Company: {company}, Sector: {sector}, Industry: {industry}, "
            f"Provider: {provider}, Date: {date}{sim_txt})"
        )
    return "\n\n".join(out)

def prepare_context_v2(inputs, lookup):
    """Build {question, context, snippets} with enriched metadata, for the v2 chain."""
    docs = inputs["docs"]
    context_text = format_docs_v2(docs, lookup)
    snippets = []
    for d in docs:
        m = enrich_meta(d.metadata, lookup)
        # normalize date to str for display
        if "PUBLICATION_DATE" in m and not isinstance(m["PUBLICATION_DATE"], str):
            try: m["PUBLICATION_DATE"] = str(m["PUBLICATION_DATE"])
            except: pass
        snippets.append({"text": d.page_content, "metadata": m})
    return {"question": inputs["question"], "context": context_text, "snippets": snippets}


In [28]:
prompt_v2 = ChatPromptTemplate.from_template(
    """You are a financial research assistant.
Use ONLY the provided news snippets and their metadata (ticker, company, sector, industry) to answer.
If the context is insufficient, say so explicitly. Cite snippets as [Doc i].

Question: {question}

Context:
{context}

Answer (concise, factual, with [Doc i] citations):"""
)


In [29]:
def make_rag_chain_v2(lookup):
    return (
        {"question": RunnablePassthrough(), "docs": retriever}
        | RunnableLambda(lambda x: prepare_context_v2(x, lookup))
        | {
            "answer": ({"question": RunnablePassthrough(), "context": RunnablePassthrough()}
                       | prompt_v2
                       | llm
                       | StrOutputParser()),
            "snippets": RunnableLambda(lambda x: x["snippets"])
        }
    )

rag_chain_v2 = make_rag_chain_v2(ticker_lookup)


In [30]:
#Function to run a list of questions and print answers in Markdown format from both versions of the chain
def run_and_compare(question_list, title, chain_v1, chain_v2):
    print_markdown(f"# {title}")
    for q in question_list:
        out1 = chain_v1.invoke(q)
        out2 = chain_v2.invoke(q)

        if isinstance(out1, str):
            ans1, snips1 = out1, []
        else:
            ans1, snips1 = out1.get("answer",""), out1.get("snippets",[])
        ans2, snips2 = out2["answer"], out2["snippets"]

        # quick diagnostics
        uniq_tickers_v1 = len({(s["metadata"] or {}).get("TICKER") for s in snips1})
        uniq_tickers_v2 = len({(s["metadata"] or {}).get("TICKER") for s in snips2})
        sectors_v2 = { (s["metadata"] or {}).get("SECTOR") for s in snips2 } - {None}

        print_markdown(f"## Q: {q}")
        print_markdown(f"**V1 Answer**\n\n{ans1}\n\n**V1 Snippets**\n\n{snippets_to_markdown(snips1)}")
        print_markdown(f"**V2 Answer (with financial metadata)**\n\n{ans2}\n\n**V2 Snippets**\n\n{snippets_to_markdown(snips2)}")
        print_markdown(f"_Coverage hint_: V1 unique tickers={uniq_tickers_v1} • V2 unique tickers={uniq_tickers_v2} • V2 sectors seen={', '.join([s for s in sectors_v2 if s]) or '—'}")


In [31]:
run_and_compare(questions_topic,   "Topic Questions — V1 vs V2", rag_chain, rag_chain_v2)
run_and_compare(questions_company, "Company Questions — V1 vs V2", rag_chain, rag_chain_v2)
run_and_compare(questions_industry,"Industry Questions — V1 vs V2", rag_chain, rag_chain_v2)

# Topic Questions — V1 vs V2

## Q: What are the major concerns expressed in financial news about inflation?

**V1 Answer**

The major concerns expressed in financial news about inflation include:

1. Persistent US inflation and its potential to lead to an economic slowdown, as highlighted in the Federal Reserve's May policy meeting [Doc 1].
2. Food inflation dampening hopes for a rate cut, indicating that rising food prices are complicating monetary policy decisions [Doc 2], [Doc 3], [Doc 4], [Doc 5].

**V1 Snippets**

**[Doc 1]** *(Ticker: BLK, Provider: Yahoo Finance UK, Date: 2025-05-29, sim=0.577)*

> Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.

**[Doc 2]** *(Ticker: TSLA, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 3]** *(Ticker: NVDA, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 4]** *(Ticker: LULU, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 5]** *(Ticker: AVGO, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead


**V2 Answer (with financial metadata)**

The major concerns expressed in financial news about inflation include:

1. Persistent US inflation and the potential for economic slowdown, as highlighted by the Federal Reserve's May policy meeting [Doc 1].
2. Food inflation dampening hopes for a rate cut, indicating ongoing inflationary pressures that could affect monetary policy decisions [Doc 2, Doc 3, Doc 4, Doc 5].

**V2 Snippets**

**[Doc 1]** *(Ticker: BLK, Provider: Yahoo Finance UK, Date: 2025-05-29, sim=0.577)*

> Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.

**[Doc 2]** *(Ticker: TSLA, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 3]** *(Ticker: NVDA, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 4]** *(Ticker: LULU, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

**[Doc 5]** *(Ticker: AVGO, Provider: Yahoo Finance UK, Date: 2025-05-31, sim=0.492)*

> The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead


_Coverage hint_: V1 unique tickers=5 • V2 unique tickers=5 • V2 sectors seen=Financial Services, Technology, Consumer Cyclical

...

_Coverage hint_: V1 unique tickers=5 • V2 unique tickers=5 • V2 sectors seen=Basic Materials, Consumer Cyclical

## Q: What risks or challenges are discussed in recent news about the energy industry?

**V1 Answer**

Recent news about the energy industry highlights several risks and challenges:

1. **Legislative Risks for Renewable Energy**: A bill advancing in Congress could significantly harm the renewable energy sector by repealing crucial subsidies. This change may render many renewable energy projects uneconomical, leading to a crash in renewable energy stocks as companies have recently increased production and generation capabilities in the U.S. [Doc 1], [Doc 2].

2. **Oilfield Service Challenges**: Companies in the oilfield service sector, such as SLB, HAL, and BKR, are facing difficulties due to sliding oil prices, rising tariffs, and shrinking drilling budgets. There is uncertainty about whether demand for liquefied natural gas (LNG) and artificial intelligence (AI) can provide sufficient support to mitigate these challenges [Doc 3], [Doc 4].

These factors indicate a turbulent environment for both renewable and traditional energy sectors.

**V1 Snippets**

**[Doc 1]** *(Ticker: NEE, Provider: Motley Fool, Date: 2025-05-23, sim=0.536)*

> Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

**[Doc 2]** *(Ticker: ENPH, Provider: Motley Fool, Date: 2025-05-23, sim=0.536)*

> Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

**[Doc 3]** *(Ticker: HAL, Provider: Zacks, Date: 2025-05-21, sim=0.482)*

> Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

**[Doc 4]** *(Ticker: BKR, Provider: Zacks, Date: 2025-05-21, sim=0.482)*

> Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

**[Doc 5]** *(Ticker: FCX, Provider: Motley Fool, Date: 2025-05-21, sim=0.478)*

> 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.


**V2 Answer (with financial metadata)**

Recent news highlights several risks and challenges in the energy industry:

1. **Legislative Risks for Renewable Energy**: A bill advancing in Congress could significantly harm the renewable energy sector by repealing crucial subsidies, making many projects uneconomical. This has led to a sharp decline in renewable energy stocks as companies had just ramped up production and generation capabilities [Doc 1][Doc 2].

2. **Market Challenges for Oilfield Services**: Companies in the oilfield service sector, such as Halliburton and Baker Hughes, are facing difficulties due to sliding oil prices, rising tariffs, and shrinking drilling budgets. These factors pose significant challenges to their profitability and operational viability [Doc 3][Doc 4]. 

These snippets indicate a turbulent environment for both renewable and traditional energy sectors, driven by legislative changes and market dynamics.

**V2 Snippets**

**[Doc 1]** *(Ticker: NEE, Provider: Motley Fool, Date: 2025-05-23, sim=0.536)*

> Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

**[Doc 2]** *(Ticker: ENPH, Provider: Motley Fool, Date: 2025-05-23, sim=0.536)*

> Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.

**[Doc 3]** *(Ticker: HAL, Provider: Zacks, Date: 2025-05-21, sim=0.482)*

> Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

**[Doc 4]** *(Ticker: BKR, Provider: Zacks, Date: 2025-05-21, sim=0.482)*

> Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?

**[Doc 5]** *(Ticker: FCX, Provider: Motley Fool, Date: 2025-05-21, sim=0.478)*

> 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.


_Coverage hint_: V1 unique tickers=5 • V2 unique tickers=5 • V2 sectors seen=Energy, Basic Materials, Technology, Utilities

## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



| Criteria       | WITHOUT METADATA | WITH METADATA |
|----------------|------------------|---------------|
| Clarity        | Clear summaries but sometimes generic; lacks company identifiers, which can make references less precise. | Equally clear but adds company names, sectors, and industries, making references more precise and easier to follow. |
| Detail & Depth | Covers key points well but omits extra identifiers and market context; less depth on company specifics. | Adds depth with sector/industry info and exact company names, giving richer context to each reference. |
| Use of Context | Uses only textual content from snippets; context is sometimes weaker when entities aren’t explicitly named. | Stronger use of context by linking facts to specific companies and industries, enhancing grounding and reader understanding. |
| Accuracy & Grounding | Accurate in summarizing snippet content, but grounding is weaker without explicit entity data. | Maintains accuracy while improving grounding through explicit metadata connections to companies/sectors. |
| Relevance      | Stays relevant to the question, but without metadata some connections (e.g., to industries) are implied, not explicit. | Stays relevant and reinforces relevance by tying themes directly to named companies and industry classifications. |
| Narrative Flow | Logical flow but can feel slightly repetitive in phrasing; transitions rely mostly on snippet content. | Flow is similar but smoother when metadata is used, since named entities and industries make transitions more natural and coherent. |


We can see that the the model improves significantly if we add metadata to the context that is given to the LLM. We could see, for example, that the Amazon question is answered in a more detailed and accurate way than with the first RAG model.

With the first model, the LLM answered that it didn't have enough context (or understanding) to answer the question; however, with the second model, although the retrieved documents were the same, the LLM tied the question with the articles and the additional metadata to make a match between Amazon, AI and logistics (sector).

With all the mentioned before, we could conclude that the model can provide better answers if fed with a richer or more complete context. We, as Data Scientists (or Data Analysts) should pay special attention to prompt engineering, document retrieval, chunking and context generation to improve our models in a production environment.